In [1]:
from netCDF4 import Dataset
import datetime
import numpy as np
import datetime as dt
import os.path

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def worst_qc(qcs):
    # given a tuple of qcs, return a list corresponding to the worst one.
    qcs = np.concatenate(qcs).reshape([len(qcs),qcs[0].size])
    qc = np.empty(shape=qcs.shape[1])
    for i in range(qcs.shape[1]):
        if np.any(qcs[:,i]==6): qc[i]=6
        elif np.any(qcs[:,i]==0): qc[i]=9
        elif np.any(qcs[:,i]==4): qc[i]=4
        elif np.any(qcs[:,i]==3): qc[i]=3
        elif np.any(qcs[:,i]==2): qc[i]=2
        else: qc[i]=1
        

In [4]:
def get_global_attributes(date_created, date_modified=' ', history='Data generated from raw glider files', metadata_link='N/A', references=' ', wmo_id=' '):
    deployment_time = '20141208'
    if date_modified==' ':
        date_modified = date_created
    global_attributes = {'Conventions' : 'CF-1.6', \
                     'Metadata_Conventions' : 'CF-1.6, Unidata Dataset Discovery v1.0', \
                     'acknowledgeent' : 'Funding from NSF OPP-1246460', \
                     'comment' : 'Data provided as is', \
                     'contributor_name' : 'Andrew F Thompson, Janet Sprintall, Zachary K Erickson, Xiaozhou Ruan, Giuliana Viglione', \
                     'contributor_role' : 'Principal Investigator, Co-Investigator, Researcher, Researcher, Researcher', \
                     'creator_email' : 'andrewt@caltech.edu', \
                     'creator_name' : 'Andrew F Thompson', \
                     'creator_url' : 'http://web.gps.caltech.edu/~andrewt/', \
                     'date_created' : date_created, \
                     'date_issued' : datetime.datetime.now().isoformat(), \
                     'date_modified' : date_modified, \
                     'format_version' : 'IOOS_Glider_NetCDF_v2.0.nc', \
                     'history': history, \
                     'id' : 'sg539-'+deployment_time, \
                     'institution' : 'California Institute of Technology', \
                     'keywords' : 'AUVS > Autonomous Underwater Vehicles, Oceans > Ocean Chemistry > Oxygen, Oceans > Ocean Optics > Fluorescence, Oceans > Ocean Optics > Gelbstoff, Oceans > Ocean Optics > Scattering, Oceans > Ocean Pressure > Water pressure, Oceans > Ocean Temperature > Water temperature, Oceans > Salinity/Density > Conductivity', \
                     'keywords_vocabulary' : 'GMCD Science Keywords', \
                     'license' : 'This data may be redistributed and used with an appropriate citation. We ask that users interested in this data for research first contact the Principal Investigator.', \
                     'metadata_link' : metadata_link, \
                     'naming_authority' : 'edu.caltech.ese', \
                     'platform_type' : 'Seaglider', \
                     'processing_level' : 'Data provided from Seaglider with minimal post-processing. Data provided as is with no expressed or implied assurance of quality assurance or quality.', \
                     'project' : 'ChinStrAP', \
                     'publisher_email' : 'zerickso@caltech.edu', \
                     'publisher_name' : 'Zachary K Erickson', \
                     'publisher_url' : 'http://web.gps.caltech.edu/~zerickso/', \
                     'references' : references, \
                     'sea_name' : 'Drake Passage, Southern Oceans (> 60 degrees South)', \
                     'source' : 'Observational data from a profiling glider', \
                     'standard_name_vocabulary' : 'CF Standard Name Table v27', \
                     'summary' : 'Seaglider dataset gathered as part of the ChinStrAP (Changes in Stratification at the Antarctic Peninsula) study, funded by NSF through OPP-1246460. This dataset contains physical oceanographic measurements of temperature, conductivity, pressure, and estimates of depth-average currents; biological oceanographic measurements of fluorescence, backscatter at 650 nm, and gelbstoff/CDOM (colored dissolved organic matter); and chemical oceanographic measurements of oxygen concentration.', \
                     'title' : 'sg539-'+deployment_time, \
                     'wmo_id' : wmo_id}
    return global_attributes

In [5]:
folder = '/Users/Zach/Documents/Cruises/chinstrap/sg539/'
filename = folder+'p539%04d.nc'
dives = range(1,643); # set of numbers from 1 to 642 (naming convention or glider dives is 1-indexed)

In [6]:
var_names = ['depth_avg_curr_north','depth_avg_curr_east','start_of_climb_time','time','theta','temperature_raw_qc','temperature_raw','temperature_qc','temperature','sigma_theta','sigma_t','salinity_raw_qc','salinity_raw','salinity_qc','salinity','pressure','log_gps_time','log_gps_lon','log_gps_lat','log_GPS2','log_GPS1','log_GPS','eng_wlbbfl2_FL2sig','eng_wlbbfl2_FL2ref','eng_wlbbfl2_FL1sig','eng_wlbbfl2_FL1ref','eng_wlbbfl2_BB1sig','eng_wlbbfl2_BB1ref','eng_elaps_t','eng_depth','eng_aa4330_O2','depth_avg_curr_qc','depth','density','ctd_time','ctd_depth','conductivity_raw_qc','conductivity_raw','conductivity_qc','conductivity','aanderaa4330_results_time','aanderaa4330_qc','aanderaa4330_instrument_dissolved_oxygen','aanderaa4330_dissolved_oxygen_qc','aanderaa4330_dissolved_oxygen','GPSE_qc','GPS2_qc','GPS1_qc','CTD_qc']


## Time-series variables

In [7]:
var_trajectory = {'cf_role' : 'trajectory_id', \
                  'comment' : 'A trajectory is a single deployment of a glider and may span multiple data files.', \
                  'long_name' : 'Trajectory/Deployment Name'}

var_time = {'ancillary_variables' : 'time_qc', \
            'calendar' : 'gregorian', \
            'long_name' : 'Time', \
            'observation_type' : 'measured', \
            'standard_name' : 'time', \
            'units' : 'seconds since 1970-01-01T00:00:00Z'}

var_time_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
               'flag_values' : '0 1 2 3 4 5 6 8 9', \
               'long_name' : 'time Quality Flag', \
               'standard_name' : 'time status_flag', \
               #'valid_max' : 9, \
               #'valid_min' : 0
              }

var_lat = {'ancillary_variables' : 'lat_qc', \
           'comment' : 'Values are interpolated from GPS fixes', \
           'coordinate_reference_frame' : 'urn:ogc:crs:EPSG::4326', \
           'long_name' : 'GPS latitude', \
           'observation_type' : 'measured', \
           'platform' : 'Seaglider', \
           'reference' : 'WGS84', \
           'standard_name' : 'latitude', \
           'units' : 'degrees_north', \
           'valid_max' : 90, \
           'valid_min' : -90}

var_lat_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
              'flag_values' : '0 1 2 3 4 5 6 8 9', \
              'long_name' : 'latitude Quality Flag', \
              'standard_name' : 'latitude status_flag', \
              #'valid_max' : 9, \
              #'valid_min' : 0
             }

var_lon = {'ancillary_variables' : 'lon_qc', \
           'comment' : 'Values are interpolated from GPS fixes', \
           'coordinate_reference_frame' : 'urn:ogc:crs:EPSG::4326', \
           'long_name' : 'GPS longitude', \
           'observation_type' : 'measured', \
           'platform' : 'Seaglider', \
           'reference' : 'WGS84', \
           'standard_name' : 'longitude', \
           'units' : 'degrees_east', \
           'valid_max' : 180, \
           'valid_min' : -180}

var_lon_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
              'flag_values' : '0 1 2 3 4 5 6 8 9', \
              'long_name' : 'latitude Quality Flag', \
              'standard_name' : 'latitude status_flag', \
              #'valid_max' : 9, \
              #'valid_min' : 0
             }

var_pressure = {'accuracy' : ' ', \
                'ancillary_variables' : 'pressure_qc', \
                'comment' : ' ', \
                'instrument' : 'instrument_ctd', \
                'long_name' : 'Pressure', \
                'observation_type' : 'measured', \
                'platform' : 'Seaglider', \
                'positive' : 'down', \
                'precision' : ' ', \
                'reference_datum' : 'sea-surface', \
                'resolution' : ' ', \
                'standard_name' : 'sea_water_pressure', \
                'units' : 'dbar', \
                'valid_max' : 2000, \
                'valid_min' : 0}

var_pressure_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                   'flag_values' : '0 1 2 3 4 5 6 8 9', \
                   'long_name' : 'pressure Quality Flag', \
                   'standard_name' : 'sea_water_pressure status_flag', \
                   #'valid_max' : 9, \
                   #'valid_min' : 0
                  }

var_depth = {'accuracy' : ' ', \
                'ancillary_variables' : 'depth_qc', \
                'comment' : ' ', \
                'instrument' : 'instrument_ctd', \
                'long_name' : 'Depth', \
                'observation_type' : 'calculated', \
                'platform' : 'platform', \
                'positive' : 'down', \
                'precision' : ' ', \
                'reference_datum' : 'sea-surface', \
                'resolution' : ' ', \
                'standard_name' : 'sea_water_pressure', \
                'units' : 'm', \
                'valid_max' : 2000, \
                'valid_min' : 0}

var_depth_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                'flag_values' : '0 1 2 3 4 5 6 8 9', \
                'long_name' : 'depth Quality Flag', \
                'standard_name' : 'depth status_flag', \
                #'valid_max' : 9, \
                #'valid_min' : 0
               }

var_temperature = {'accuracy' : '0.01', \
                   'ancillary_variables' : 'temperature_qc', \
                   'instrument' : 'Seabird SBE3', \
                   'long_name' : 'Temperature', \
                   'observation_type' : 'measured', \
                   'platform' : 'Seaglider', \
                   'precision' : '0.001', \
                   'resolution' : ' ', \
                   'standard_name' : 'sea_water_temperature', \
                   'units' : 'degrees Celsius', \
                   'valid_max' : 40, \
                   'valid_min' : -3}

var_temperature_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                      'flag_values' : '0 1 2 3 4 5 6 8 9', \
                      'long_name' : 'temperature Quality Flag', \
                      'standard_name' : 'sea_water_temperature status_flag', \
                      #'valid_max' : 9,
                      #'valid_min' : 0
                     }

var_conductivity = {'accuracy' : ' ', \
                    'ancillary_variables' : 'conductivity_qc', \
                    'instrument' : 'Seabird SBE4', \
                    'long_name' : 'Conductivity', \
                    'observation_type' : 'measured', \
                    'platform' : 'Seaglider', \
                    'precision' : '0.0003', \
                    'resolution' : ' ', \
                    'standard_name' : 'sea_water_electrical_conductivity', \
                    'units' : 'S/m', \
                    'valid_max' : 10, \
                    'valid_min' : 0}

var_conductivity_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                       'flag_values' : '0 1 2 3 4 5 6 8 9', \
                       'long_name' : 'conductivity Quality Flag', \
                       'standard_name' : 'sea_water_electrical_conductivity status_flag', \
                       #'valid_max' : 9,
                       #'valid_min' : 0
                      }

var_salinity = {'accuracy' : '0.01', \
                'ancillary_variables' : 'salinity_qc', \
                'instrument' : 'Seabird SBE4', \
                'long_name' : 'salinity', \
                'observation_type' : 'calculated', \
                'platform' : 'Seaglider', \
                'precision' : '0.001', \
                'resolution' : ' ', \
                'standard_name' : 'sea_water_practical_salinity', \
                'units' : 'PSU', \
                'valid_max' : 40., \
                'valid_min' : 0.}

var_salinity_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                   'flag_value' : '0 1 2 3 4 5 6 8 9', \
                   'long_name' : 'salinity Quality Flag', \
                   'standard_name' : 'sea_water_practical_salinity status_flag', \
                   #'valid_max' : 9,
                   #'valid_min' : 0
                  }

var_density = {'accuracy' : ' ', \
               'ancillary_variables' : 'density_qc', \
               'instrument' : ' ', \
               'long_name' : 'density', \
               'observation_type' : 'calculated', \
               'platform' : 'Seaglider', \
               'precision' : ' ', \
               'resolution' : ' ', \
               'standard_name' : 'sea_water_density', \
               'units' : 'kg/m3', \
               'valid_max' : 1040., \
               'valid_min' : 1015.}

var_density_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                  'flag_values' : '0 1 2 3 4 5 6 8 9', \
                  'long_name' : 'density Quality Flag', \
                  'standard_name' : 'sea_water_density status_flag', \
                  #'valid_max' : 9,
                  #'valid_min' : 0
                 }

var_oxygen = {'accuracy' : '2', \
              'ancillary_variables' : 'oxygen_qc', \
              'instrument' : 'Aanderaa 4330F oxygen optode', \
              'long_name' : 'Oxygen', \
              'observation_type' : 'measured', \
              'platform' : 'Seaglider', \
              'precision' : ' ', \
              'resolution' : ' ', \
              'standard_name' : 'volume_fraction_of_oxygen_in_sea_water', \
              'units' : 'micromoles per kg', \
              'valid_max' : 300., \
              'valid_min' : 0.}

var_oxygen_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                 'flag_values' : '0 1 2 3 4 5 6 8 9', \
                 'long_name' : 'Oxygen Quality Flag', \
                 'standard_name' : 'volume_fraction_of_oxygen_in_sea_water status_flag', \
                 #'valid_max' : 9,
                 #'valid_min' : 0
                }

var_fluorescence = { \
                    'accuracy' : ' ', \
                    'ancillary_variables' : 'fluorescence_qc', \
                    'instrument' : 'WET Labs ECO Chlorophyll fluorometer', \
                    'long_name' : 'Chlorophyll fluorescence', \
                    'observation_type' : 'measured', \
                    'platform' : 'Seaglider', \
                    'precision' : ' ', \
                    'resolution' : '1.1', \
                    'standard_name' : 'mass_fraction_of_chlorophyll_a_in_sea_water', \
                    'units' : 'Counts', \
                    'valid_max' : 4130., \
                    'valid_min' : 0., \
                    'wavelength_excited' : '470', \
                    'wavelength_measured' : '695', \
                    'factory_scale_factor' : '0.0129', \
                    'factory_dark_counts' : '49', \
                    'calibration_temperature' : '22.3', \
                    'history' : ' ', \
                    'comment' : 'Scale factor is in units of micrograms/liter/count, and was determined using a mono-culture of Thalassiosira weissflogii. Extensive documentation can be found at: http://www.seabird.com/sites/default/files/documents/ECOPuckben.pdf'}

var_fluorescence_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                       'flag_values' : '0 1 2 3 4 5 6 8 9', \
                       'long_name' : 'Chlorophyll fluorescence Quality Flag', \
                       'standard_name' : 'mass_fraction_of_chlorophyll_a_in_sea_water status_flag', \
                       #'valid_max' : 9,
                       #'valid_min' : 0
                      }

var_backscatter = {'accuracy' : ' ', \
                   'ancillary_variables' : 'backscatter_qc', \
                   'instrument' : 'WET Labs Scattering meter', \
                   'long_name' : 'Optical Backscatter', \
                   'observation_type' : 'measured', \
                   'platform' : 'Seaglider', \
                   'precision' : ' ', \
                   'resolution' : '1.0', \
                   'standard_name' : ' ', \
                   'valid_max' : 200., \
                   'valid_min' : 0., \
                   'wavelength_measured' : '650 nm', \
                   'factory_scale_factor' : '4.32e-6', \
                   'factory_dark_counts' : '42', \
                   'centroid_angle' : 124, \
                   'history' : ' ', \
                   'comment' : 'For extensive documentation, see http://www.seabird.com/sites/default/files/documents/ECOPuckben.pdf'}

var_backscatter_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                      'flag_values' : '0 1 2 3 4 5 6 8 9', \
                      'long_name' : 'Optical backscatter Quality Flag', \
                      'standard_name' : ' status_flag', \
                      #'valid_max' : 9,
                      #'valid_min' : 0
                     }

var_CDOM = {'accuracy' : ' ', \
            'ancillary_variables' : 'CDOM_qc', \
            'instrument' : 'WET Labs ECO CDOM Fluorometer', \
            'long_name' : 'Chromophoric Dissolved Organic Matter', \
            'observation_type' : 'measured', \
            'platform' : 'Seaglider', \
            'precision' : ' ', \
            'resolution' : '1.1', \
            'standard_name' : 'concentration_of_colored_dissolved_organic_matter_in_sea_water_expressed_as_equivalent_mass_fraction_of_quinine_sulfate_dihydrate', \
            'units' : 'Counts', \
            'valid_max' : 4130., \
            'valid_min' : 0., \
            'wavelength_excited' : '370', \
            'wavelength_measured' : '460', \
            'factory_scale_factor' : '0.1661', \
            'factory_dark_counts' : '34', \
            'calibration_temperature' : '22.3', \
            'history' : ' ', \
            'comment' : 'Scale factor is in units of ppb/count. Extensive documentation can be found at: http://www.seabird.com/sites/default/files/documents/ECOPuckben.pdf, where this parameter is called "Fluorescence Dissolved Organic Matter (FDOM)"'}

var_CDOM_qc = {'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
               'flag_values' : '0 1 2 3 4 5 6 8 9', \
               'long_name' : 'Chromophoric Dissolved Organic Matter Quality Flag', \
               'standard_name' : 'concentration_of_colored_dissolved_organic_matter_in_sea_water_expressed_as_equivalent_mass_fraction_of_quinine_sulfate_dihydrate status_flag', \
               #'valid_max' : 9,
               #'valid_min' : 0
              }

## Dimensionless Profile Variables

In [8]:
var_profile_id = { \
                  'comment' : 'Sequential profile number within the trajectory. This value is unique in each file that is part of a single trajectory/deployment.', \
                  'long_name' : 'Profile ID', \
                  'valid_max' : 2000, \
                  'valid_min' : 1}

var_profile_time = { \
                    'calendar' : 'gregorian', \
                    'comment' : 'Timestamp corresponding to the mid-point of the profile', \
                    'long_name' : 'Profile Center Time', \
                    'observation_type' : 'calculated', \
                    'platform' : 'Seaglider', \
                    'standard_name' : 'time', \
                    'units' : 'seconds since 1970-01-01T00:00:00Z'}

var_profile_time_qc = { \
                       'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                       'flag_values' : '0 1 2 3 4 5 6 8 9', \
                       'long_name' : 'profile_time Quality Flag', \
                       'standard_name' : 'time status_flag', \
                       #'valid_max' : 9,
                       #'valid_min' : 0
                      }

var_profile_lat = { \
                   'comment' : 'Value is interpolated to provide an estimate of the latitude at the mid-point of the profile.', \
                   'long_name' : 'Profile Center Latitude', \
                   'observation_type' : 'calculated', \
                   'platform' : 'Seaglider', \
                   'standard_name' : 'latitude', \
                   'units' : 'degrees_north', \
                   'valid_max' : 90., \
                   'valid_min' : -90.}

var_profile_lat_qc = { \
                      'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                      'flag_values' : '0 1 2 3 4 5 6 8 9', \
                      'long_name' : 'profile_lat Quality Flag', \
                      'standard_name' : 'latitude status_flag', \
                      #'valid_max' : 9,
                      #'valid_min' : 0
                     }

var_profile_lon = { \
                   'comment' : 'Value is interpolated to provide an estimate of the longitude at the mid-point of the profile.', \
                   'long_name' : 'Profile Center Longitude', \
                   'observation_type' : 'calculated', \
                   'platform' : 'Seaglider', \
                   'standard_name' : 'longitude', \
                   'units' : 'degrees_east', \
                   'valid_max' : 180., \
                   'valid_min' : -180.}

var_profile_lon_qc = { \
                      'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                      'flag_values' : '0 1 2 3 4 5 6 8 9', \
                      'long_name' : 'profile_lon Quality Flag', \
                      'standard_name' : 'longitude status_flag', \
                      #'valid_max' : 9,
                      #'valid_min' : 0
                     }

var_time_uv = { \
               'calendar' : 'gregorian', \
               'comment' : 'The depth-averaged current is an estimate of the net current measured while the glider is underwater.  The value is calculated over the entire underwater segment, which may consist of 1 or more dives.', \
               'long_name' : 'Depth-Averaged Time', \
               'observation_type' : 'calculated', \
               'standard_name' : 'time', \
               'units' : 'seconds since 1970-01-01T00:00:00Z'}

var_time_uv_qc = { \
                  'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                  'flag_values' : '0 1 2 3 4 5 6 8 9', \
                  'long_name' : 'time_uv Quality Flag', \
                  'standard_name' : 'time status_flag', \
                  #'valid_max' : 9,
                  #'valid_min' : 0
                 }

var_lat_uv = { \
              'comment' : 'The depth-averaged current is an estimate of the net current measured while the glider is underwater.  The value is calculated over the entire underwater segment, which may consist of 1 or more dives.', \
              'long_name' : 'Depth-Averaged Latitude', \
              'observation_type' : 'calculated', \
              'platform' : 'platform', \
              'standard_name' : 'latitude', \
              'units' : 'degrees_north', \
              'valid_max' : 90., \
              'valid_min' : -90.}

var_lat_uv_qc = { \
                 'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                 'flag_values' : '0 1 2 3 4 5 6 8 9', \
                 'long_name' : 'lat_uv Quality Flag', \
                 'standard_name' : 'latitude status_flag', \
                 #'valid_max' : 9,
                 #'valid_min' : 0
                }

var_lon_uv = { \
              'comment' : 'The depth-averaged current is an estimate of the net current measured while the glider is underwater.  The value is calculated over the entire underwater segment, which may consist of 1 or more dives.', \
              'long_name' : 'Depth-Averaged Longitude', \
              'observation_type' : 'calculated', \
              'platform' : 'platform', \
              'standard_name' : 'longitude', \
              'units' : 'degrees_east', \
              'valid_max' : 180., \
              'valid_min' : -180.}

var_lon_uv_qc = { \
                 'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                 'flag_values' : '0 1 2 3 4 5 6 8 9', \
                 'long_name' : 'lon_uv Quality Flag', \
                 'standard_name' : 'longitude status_flag', \
                 #'valid_max' : 9,
                 #'valid_min' : 0
                }

var_u = { \
         'comment' : 'The depth-averaged current is an estimate of the net current measured while the glider is underwater.  The value is calculated over the entire underwater profile (downcast and upcast combined).', \
         'long_name' : 'Depth-averaged eastward sea water velocity', \
         'observation_type' : 'calculated', \
         'platform' : 'Seaglider', \
         'standard_name' : 'eastward_sea_water_velocity', \
         'units' : 'm/s', \
         'valid_max' : 10, \
         'valid_min' : -10}

var_u_qc = { \
            'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
            'flag_values' : '0 1 2 3 4 5 6 8 9', \
            'long_name' : 'u Quality Flag', \
            'standard_name' : 'eastward_sea_water_velocity status_flag', \
            #'valid_max' : 9,
            #'valid_min' : 0
           }

var_v = { \
         'comment' : 'The depth-averaged current is an estimate of the net current measured while the glider is underwater.  The value is calculated over the entire underwater profile (downcast and upcast combined).', \
         'long_name' : 'Depth-averaged northward sea water velocity', \
         'observation_type' : 'calculated', \
         'platform' : 'Seaglider', \
         'standard_name' : 'northward_sea_water_velocity', \
         'units' : 'm/s', \
         'valid_max' : 10, \
         'valid_min' : -10}

var_v_qc = { \
            'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
            'flag_values' : '0 1 2 3 4 5 6 8 9', \
            'long_name' : 'v Quality Flag', \
            'standard_name' : 'northward_sea_water_velocity status_flag', \
            #'valid_max' : 9,
            #'valid_min' : 0
           }

var_time_uv_surf = { \
                    'calendar' : 'gregorian', \
                    'comment' : 'The surface current is an estimate of the net current measured while the glider is at the surface.', \
                    'long_name' : 'Surface-Averaged Time', \
                    'observation_type' : 'calculated', \
                    'standard_name' : 'time', \
                    'units' : 'seconds since 1970-01-01T00:00:00Z'}

var_time_uv_surf_qc = { \
                       'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                       'flag_values' : '0 1 2 3 4 5 6 8 9', \
                       'long_name' : 'time_uv_surf Quality Flag', \
                       'standard_name' : 'time status_flag', \
                       #'valid_max' : 9,
                       #'valid_min' : 0
                      }

var_lat_uv_surf = { \
                   'comment' : 'The surface current is an estimate of the net current measured while the glider is at the surface.', \
                   'long_name' : 'Surface-Averaged Latitude', \
                   'observation_type' : 'calculated', \
                   'standard_name' : 'latitude', \
                   'units' : 'degrees_north', \
                   'valid_max' : 90., \
                   'valid_min' : -90.}

var_lat_uv_surf_qc = { \
                      'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                      'flag_values' : '0 1 2 3 4 5 6 8 9', \
                      'long_name' : 'lat_uv_surf Quality Flag', \
                      'standard_name' : 'latitude status_flag', \
                      #'valid_max' : 9,
                      #'valid_min' : 0
                     }

var_lon_uv_surf = { \
                   'comment' : 'The surface current is an estimate of the net current measured while the glider is at the surface.', \
                   'long_name' : 'Surface-Averaged Longitude', \
                   'observation_type' : 'calculated', \
                   'standard_name' : 'longitude', \
                   'units' : 'degrees_east', \
                   'valid_max' : 180., \
                   'valid_min' : -180.}

var_lon_uv_surf_qc = { \
                      'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                      'flag_values' : '0 1 2 3 4 5 6 8 9', \
                      'long_name' : 'lon_uv_surf Quality Flag', \
                      'standard_name' : 'longitude status_flag', \
                      #'valid_max' : 9,
                      #'valid_min' : 0
                     }

var_u_surf = { \
              'comment' : 'The surface current is an estimate of the net current measured while the glider is at the surface.', \
              'long_name' : 'Surface-averaged eastward sea water velocity', \
              'observation_type' : 'calculated', \
              'platform' : 'Seaglider', \
              'standard_name' : 'eastward_sea_water_velocity', \
              'units' : 'm/s', \
              'valid_max' : 10, \
              'valid_min' : -10}

var_u_surf_qc = { \
                 'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                 'flag_values' : '0 1 2 3 4 5 6 8 9', \
                 'long_name' : 'u_surf Quality Flag', \
                 'standard_name' : 'eastward_sea_water_velocity status_flag', \
                 #'valid_max' : 9,
                 #'valid_min' : 0
                }

var_v_surf = { \
              'comment' : 'The surface current is an estimate of the net current measured while the glider is at the surface.', \
              'long_name' : 'Surface-averaged northward sea water velocity', \
              'observation_type' : 'calculated', \
              'platform' : 'Seaglider', \
              'standard_name' : 'northward_sea_water_velocity', \
              'units' : 'm/s', \
              'valid_max' : 10, \
              'valid_min' : -10}

var_v_surf_qc = { \
                 'flag_meanings' : 'QC_NO_CHANGE QC_GOOD QC_PROBABLY_GOOD QC_PROBABLY_BAD QC_BAD QC_CHANGED QC_UNSAMPLED QC_INTERPOLATED QC_MISSING', \
                 'flag_values' : '0 1 2 3 4 5 6 8 9', \
                 'long_name' : 'v_surf Quality Flag', \
                 'standard_name' : 'northward_sea_water_velocity status_flag', \
                 #'valid_max' : 9,
                 #'valid_min' : 0
                }

## Dimensionless Container Variables

In [9]:
var_platform = { \
                'comment' : 'Seaglider sg539', \
                'name' : 'Queequeg', \
                'id' : 'sg539', \
                'instrument' : 'instrument_ctd',\
                'long_name' : 'California Institute of Technology Seaglider sg539', \
                'type' : 'platform', \
                'wmo_id' : ' '}

var_instrument_ctd = { \
                      'calibration_date' : ' ', \
                      'calibration_report' : ' ', \
                      'comment' : 'unpumped CTD', \
                      'factory_calibrated' : '06/10/2012', \
                      'long_name' : 'Seaglider Payload CTD', \
                      'make_model' : ' ', \
                      'platform' : 'Seaglider', \
                      'serial_number' : '0169',
                      'type' : 'platform'}

var_instrument_oxygen = { \
                         'calibration_date' : ' ', \
                         'calibration_report' : ' ', \
                         'comment' : ' ', \
                         'factory_calibrated' : '13/02/2014', \
                         'long_name' : ' ', \
                         'make_model' : 'Oxygen Optode 4831', \
                         'platform' : 'Seaglider', \
                         'serial_number' : '326',
                         'type' : 'platform'}

var_instrument_Wetlabs = { \
                          'calibration_date' : ' ', \
                          'calibration_report' : ' ', \
                          'comment' : ' ', \
                          'factory_calibrated' : '09/10/2012', \
                          'long_name' : ' ', \
                          'make_model' : ' WET Labs ECO Puck Triplet', \
                          'platform' : 'Seaglider', \
                          'serial_number' : 'BBFL2VMT-762',
                          'type' : 'platform'}

In [10]:
timeseries_vars = { \
        'time' : var_time, \
        'time_qc' : var_time_qc, \
        'lat' : var_lat, \
        'lat_qc' : var_lat_qc, \
        'lon' : var_lon, \
        'lon_qc' : var_lon_qc, \
        'pressure' : var_pressure, \
        'pressure_qc' : var_pressure_qc, \
        'depth' : var_depth, \
        'depth_qc' : var_depth_qc, \
        'temperature' : var_temperature, \
        'temperature_qc' : var_temperature_qc, \
        'conductivity' : var_conductivity, \
        'conductivity_qc' : var_conductivity_qc, \
        'salinity' : var_salinity, \
        'salinity_qc' : var_salinity_qc, \
        'density' : var_density, \
        'density_qc' : var_density_qc, \
        'oxygen' : var_oxygen, \
        'oxygen_qc' : var_oxygen_qc, \
        'fluorescence' : var_fluorescence, \
        'fluorescence_qc' : var_fluorescence_qc, \
        'backscatter' : var_backscatter, \
        'backscatter_qc' : var_backscatter_qc, \
        'CDOM' : var_CDOM, \
        'CDOM_qc' : var_CDOM_qc
                  }

profile_vars = { \
        'profile_id' : var_profile_id, \
        'profile_time' : var_profile_time, \
        'profile_time_qc' : var_profile_time_qc, \
        'profile_lat' : var_profile_lat, \
        'profile_lat_qc' : var_profile_lat_qc, \
        'profile_lon' : var_profile_lon, \
        'profile_lon_qc' : var_profile_lon_qc, \
        'time_uv' : var_time_uv, \
        'time_uv_qc' : var_time_uv_qc, \
        'lat_uv' : var_lat_uv, \
        'lat_uv_qc' : var_lat_uv_qc, \
        'lon_uv' : var_lon_uv, \
        'lon_uv_qc' : var_lon_uv_qc, \
        'u' : var_u, \
        'u_qc' : var_u_qc, \
        'v' : var_v, \
        'v_qc' : var_v_qc, \
        'time_uv_surf' : var_time_uv_surf, \
        'time_uv_surf_qc' : var_time_uv_surf_qc, \
        'lat_uv_surf' : var_lat_uv_surf, \
        'lat_uv_surf_qc' : var_lat_uv_surf_qc,\
        'lon_uv_surf' : var_lon_uv_surf, \
        'lon_uv_surf_qc' : var_lon_uv_surf_qc, \
        'u_surf' : var_u_surf, \
        'u_surf_qc' : var_u_surf_qc, \
        'v_surf' : var_v_surf, \
        'v_surf_qc' : var_v_surf_qc
               }

container_vars = { \
        'platform' : var_platform, \
        'instrument_ctd' : var_instrument_ctd, \
        'instrument_oxygen' : var_instrument_oxygen, \
        'instrument_Wetlabs' : var_instrument_Wetlabs
                 }

In [25]:
nc.close()
NC.close()

In [26]:
print('Total dives: %d' % dives[-1])
reference_time = datetime.datetime(year=1970,month=1,day=1)
for dive in dives:
    file = filename % dive
    if not os.path.isfile(file):
        print('N', end=' ')
        continue
        
    nc_var = ()
    nc_pvar = ()
    nc_cvar = ()
        
    print(dive, end=' ')

    nc = Dataset(file)
    max_depth_index = np.argmax(nc.variables['pressure'][:])
    downcast_inds = np.arange(max_depth_index)
    upcast_inds = np.arange(max_depth_index,nc.variables['pressure'].size)
    
    GPS_time = nc.variables['log_gps_time'][:]
    GPS_lat = nc.variables['log_gps_lat'][:]
    GPS_lon = nc.variables['log_gps_lon'][:]
    
    time = nc.variables['time'][downcast_inds]
    lat = np.interp(time,GPS_time,GPS_lat)
    lon = np.interp(time,GPS_time,GPS_lon)
    # do downcast
    
    NC = Dataset('SG539_profile%03d.nc' % dive,'w')
    NC.setncatts(get_global_attributes(str(dt.datetime.now()),references='Erickson et al., GRL, 2016, doi:10.1002/2016GL070565; Ruan et al., Nat. Geosci., 2017, doi:10.1038/ngeo3053'))
    NC.createDimension('time',len(downcast_inds))
    NC.createDimension('traj_strlen',1)
    var_trajectory = NC.createVariable('trajectory','S1','traj_strlen')
    var_trajectory.cf_role = "trajectory_id";
    var_trajectory.comment = "A trajectory is a single deployment of a glider and may span multiple data files." ;
    var_trajectory.long_name = "Trajectory/Deployment Name" ;
    for key in timeseries_vars.keys():
        if key[-2::]=='qc':
            dtype = 'S1'; fv = -127;
        elif key[0:2]=='fl' or key[0:2]=='ba' or key[0:2]=='CD':
            dtype = np.dtype('int64').char; fv=-999;
        else:
            dtype = np.dtype('float64').char; fv = -999;
        nc_var += (NC.createVariable(key,dtype,('time'),fill_value=fv),)
        nc_var[-1].setncatts(timeseries_vars[key])
    for key in profile_vars.keys():
        nc_pvar += (NC.createVariable(key,np.dtype('float64').char,('traj_strlen'),fill_value=-999),)
        nc_pvar[-1].setncatts(profile_vars[key])
    for key in container_vars.keys():
        nc_cvar += (NC.createVariable(key,np.dtype('int32').char,('traj_strlen'),fill_value=-999),)
        nc_cvar[-1].setncatts(container_vars[key])

    var_trajectory[:] = 'sg539-20141208'
    # Time-series variables
    nc_var[ 0][:] = time
    nc_var[ 1][:] = np.zeros(shape=time.shape) # no qc performed on time
    nc_var[ 2][:] = lat
    nc_var[ 3][:] = 8*np.ones(shape=time.shape) # all interpolated values
    nc_var[ 4][:] = lon
    nc_var[ 5][:] = 8*np.ones(shape=time.shape) # all interpolated values
    nc_var[ 6][:] = nc.variables['pressure'][downcast_inds]
    nc_var[ 7][:] = np.zeros(shape=time.shape) # no qc performed on pressure
    nc_var[ 8][:] = nc.variables['depth'][downcast_inds]
    nc_var[ 9][:] = np.zeros(shape=time.shape) # no qc performed on depth
    if 'temperature' in nc.variables:
        nc_var[10][:] = nc.variables['temperature'][downcast_inds]
        nc_var[11][:] = nc.variables['temperature_qc'][downcast_inds]
    else:
        nc_var[10][:] = np.nan*np.empty(shape=time.shape)
        nc_var[11][:] = 9*np.ones(shape=time.shape)
        print('No temperature data', end=' ')
    if 'temperature' in nc.variables:
        nc_var[12][:] = nc.variables['conductivity'][downcast_inds]
        nc_var[13][:] = nc.variables['conductivity_qc'][downcast_inds]
    else:
        nc_var[12][:] = np.nan*np.empty(shape=time.shape)
        nc_var[13][:] = 9*np.ones(shape=time.shape)
        print('No conductivity data', end=' ')    
    if 'salinity' in nc.variables:
        nc_var[14][:] = nc.variables['salinity'][downcast_inds]
        nc_var[15][:] = nc.variables['salinity_qc'][downcast_inds]
    else:
        nc_var[14][:] = np.nan*np.empty(shape=time.shape)
        nc_var[15][:] = 9*np.ones(shape=time.shape)
        print('No salinity data', end=' ')   
    if 'density' in nc.variables:
        nc_var[16][:] = nc.variables['density'][downcast_inds]
        nc_var[17][:] = worst_qc((nc.variables['pressure'][downcast_inds],nc.variables['temperature_qc'][downcast_inds],nc.variables['salinity_qc'][downcast_inds]))
    else:
        nc_var[14][:] = np.nan*np.empty(shape=time.shape)
        nc_var[15][:] = 9*np.ones(shape=time.shape)
        print('No density data', end=' ')     
    if 'aanderaa4330_dissolved_oxygen' in nc.variables:
        nc_var[18][:] = nc.variables['aanderaa4330_dissolved_oxygen'][downcast_inds]
        nc_var[19][:] = nc.variables['aanderaa4330_dissolved_oxygen_qc'][downcast_inds]
    else:
        nc_var[14][:] = np.nan*np.empty(shape=time.shape)
        nc_var[15][:] = 9*np.ones(shape=time.shape)
        print('No oxygen data', end=' ')     
    if 'eng_wlbbfl2_FL1sig' in nc.variables:
        nc_var[20][:] = nc.variables['eng_wlbbfl2_FL1sig'][downcast_inds]
        nc_var[21][:] = np.zeros(shape=time.shape) # no qc performed on fluorescence
    else:
        nc_var[20][:] = np.nan*np.empty(shape=time.shape)
        nc_var[21][:] = 9*np.ones(shape=time.shape)
        print('No FL1 data', end=' ')
    if 'eng_wlbbfl2_BB1sig' in nc.variables:
        nc_var[22][:] = nc.variables['eng_wlbbfl2_BB1sig'][downcast_inds]
        nc_var[23][:] = np.zeros(shape=time.shape) # no qc performed on backscatter
    else:
        nc_var[22][:] = np.nan*np.empty(shape=time.shape)
        nc_var[23][:] = 9*np.ones(shape=time.shape)  
        print('No BB1 data', end=' ')
    if 'eng_wlbbfl2_FL2sig' in nc.variables:
        nc_var[24][:] = nc.variables['eng_wlbbfl2_FL2sig'][downcast_inds]
        nc_var[25][:] = np.zeros(shape=time.shape) # no qc performed on CDOM
    else:
        nc_var[24][:] = np.nan*np.empty(shape=time.shape)
        nc_var[25][:] = 9*np.ones(shape=time.shape)  
        print('No FL3 data', end=' ')

    #profile variables
    midpoint_ind = int(len(downcast_inds)/2)
    nc_pvar[ 0][:] = dive*2
    nc_pvar[ 1][:] = time[midpoint_ind]
    nc_pvar[ 2][:] = 8
    nc_pvar[ 3][:] = lat[midpoint_ind]
    nc_pvar[ 4][:] = 8
    nc_pvar[ 5][:] = lon[midpoint_ind]
    nc_pvar[ 6][:] = 8
    nc_pvar[ 7][:] = np.mean(GPS_time[1::])
    nc_pvar[ 8][:] = 8
    nc_pvar[ 9][:] = np.mean(GPS_lat[1::])
    nc_pvar[10][:] = 8
    nc_pvar[11][:] = np.mean(GPS_lon[1::])
    nc_pvar[12][:] = 8
    if 'depth_avg_curr_east' in nc.variables:
        nc_pvar[13][:] = nc.variables['depth_avg_curr_east'][:]
        nc_pvar[14][:] = nc.variables['depth_avg_curr_qc'][:][0]
        nc_pvar[15][:] = nc.variables['depth_avg_curr_north'][:]
        nc_pvar[16][:] = nc.variables['depth_avg_curr_qc'][:][0]
    else:
        nc_pvar[13][:] = np.nan
        nc_pvar[14][:] = 9
        nc_pvar[15][:] = np.nan
        nc_pvar[16][:] = 9
        print('No DAC', end=' ')
    nc_pvar[17][:] = np.mean(GPS_time[0:-1])
    nc_pvar[18][:] = 8
    nc_pvar[19][:] = np.mean(GPS_lat[0:-1])
    nc_pvar[20][:] = 8
    nc_pvar[21][:] = np.mean(GPS_lon[0:-1])
    nc_pvar[22][:] = 8
    if 'surface_curr_east' in nc.variables:
        nc_pvar[23][:] = nc.variables['surface_curr_east'][:]
        nc_pvar[24][:] = nc.variables['surface_curr_qc'][:][0]
        nc_pvar[25][:] = nc.variables['surface_curr_north'][:]
        nc_pvar[26][:] = nc.variables['surface_curr_qc'][:][0] 
    else:
        nc_pvar[23][:] = np.nan
        nc_pvar[24][:] = 9
        nc_pvar[25][:] = np.nan
        nc_pvar[26][:] = 9
        print('No surf. current', end=' ')
    
    nc.close()
    NC.close()


Total dives: 642
1 N N 4 No FL1 data No BB1 data No FL3 data 5 No FL1 data No BB1 data No FL3 data 6 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in greater
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in greater
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in greater
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys
/Library/Frameworks/Python.framework/Versions/3.6/lib/p

No FL1 data No BB1 data No FL3 data 7 No FL1 data No BB1 data No FL3 data 8 No FL1 data No BB1 data No FL3 data 9 No FL1 data No BB1 data No FL3 data 10 No FL1 data No BB1 data No FL3 data 11 12 No FL1 data No BB1 data No FL3 data 13 No FL1 data No BB1 data No FL3 data 14 No FL1 data No BB1 data No FL3 data 15 No FL1 data No BB1 data No FL3 data 16 No FL1 data No BB1 data No FL3 data 17 No FL1 data No BB1 data No FL3 data 18 No FL1 data No BB1 data No FL3 data 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 